# Strava API

https://towardsdatascience.com/using-the-strava-api-and-pandas-to-explore-your-activity-data-d94901d9bfde

https://medium.com/@lejczak.learn/get-your-strava-activity-data-using-python-2023-%EF%B8%8F-b03b176965d0

In [1]:
import requests 
from dotenv import load_dotenv
import os 
import pandas as pd 
from datetime import datetime, timedelta

## Env Variables

In [4]:
# Env Variables
load_dotenv()
client_id = os.environ["CLIENT_ID"] 
client_secret = os.environ["CLIENT_SECRET"] 
# refresh_token = os.environ["REFRESH_TOKEN"]

auth_endpoint = "https://www.strava.com/oauth/token"
activites_endpoint = "https://www.strava.com/api/v3/athlete/activities"

In [5]:
# Paste this in Web browser 
auth_path = f"https://www.strava.com/oauth/authorize?client_id={client_id}&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=activity:read_all"
auth_path

'https://www.strava.com/oauth/authorize?client_id=131915&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=activity:read_all'

In [6]:
code = "7907a1ffca215c4b432d0bf2347497db987cba60"

payload = {
        "client_id": client_id,
        "client_secret": client_secret,
        "code": code,
        "grant_type": "authorization_code",
        "f": "json"
}
res = requests.post(auth_endpoint, data=payload, verify=False)
refresh_token = res.json()['refresh_token']
refresh_token

/Users/bmoreno/miniforge3/envs/data-science/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.strava.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'c8c5c851df9ec3e2c9ee45c61d2a7f982d8d1f05'

## Login

In [7]:
import requests 

def get_access_token():
    # these params needs to be passed to get access
    # token used for retrieveing actual data
    payload = {
        "client_id": client_id,
        "client_secret": client_secret,
        'refresh_token': refresh_token,
        "grant_type": "refresh_token",
        'f': 'json'
    }
    res = requests.post(auth_endpoint, data=payload, verify=False)
    access_token = res.json()['access_token']

    return access_token

access_token = get_access_token()
print(access_token)

/Users/bmoreno/miniforge3/envs/data-science/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.strava.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


86903e13ec17c9625aa2fb41d5042359803bdfa3


In [8]:
def access_activity_data(access_token, params=None):
    
    headers = {'Authorization': f'Authorization: Bearer {access_token}'}    

    if not params:
        response = requests.get(activites_endpoint, headers=headers)
    response = requests.get(activites_endpoint, headers=headers, params=params)
    
    response.raise_for_status()
    activity_data = response.json()

    return activity_data

ACTIVITIES_PER_PAGE = 100
PAGE_NUMBER = 1

GET_ALL_ACTIVITIES_PARAMS = {
    'per_page': ACTIVITIES_PER_PAGE,
    'page': PAGE_NUMBER
}

activities = access_activity_data(access_token, params=GET_ALL_ACTIVITIES_PARAMS)
activities

[{'resource_state': 2,
  'athlete': {'id': 87979082, 'resource_state': 1},
  'name': 'Thursday Evening Weight Training',
  'distance': 0.0,
  'moving_time': 2996,
  'elapsed_time': 2996,
  'total_elevation_gain': 0,
  'type': 'WeightTraining',
  'sport_type': 'WeightTraining',
  'id': 12099390057,
  'start_date': '2024-08-08T23:36:53Z',
  'start_date_local': '2024-08-08T19:36:53Z',
  'timezone': '(GMT-04:00) America/Anguilla',
  'utc_offset': -14400.0,
  'location_city': None,
  'location_state': None,
  'location_country': None,
  'achievement_count': 0,
  'kudos_count': 0,
  'comment_count': 0,
  'athlete_count': 1,
  'photo_count': 0,
  'map': {'id': 'a12099390057', 'summary_polyline': '', 'resource_state': 2},
  'trainer': True,
  'commute': False,
  'manual': False,
  'private': False,
  'visibility': 'everyone',
  'flagged': False,
  'gear_id': None,
  'start_latlng': [],
  'end_latlng': [],
  'average_speed': 0.0,
  'max_speed': 0.0,
  'has_heartrate': True,
  'average_heartrate

In [13]:
for d in activities: 

    id = d['id']
    activity_name = d['name'] 
    activity_type = d['type']
    
    if d['type'] not in ["WeightTraining", "Workout"]:
        print(activity_name)
        print(activity_type)
        
        url = f"https://www.strava.com/api/v3/activities/{id}/streams"
        header = {'Authorization': 'Bearer ' + access_token}
        latlong = requests.get(url, headers=header, params={'keys':['latlng']}).json()[0]['data']
        altitude = requests.get(url, headers=header, params={'keys':['altitude']}).json()[1]['data']

        data_ = pd.DataFrame([*latlong], columns=['lat','long'])
        data_['altitude'] = altitude
        data_['activity_id'] = id 
        data_['activity_name'] = activity_name        
        data_['activity_type'] = activity_type    

        data_.to_csv(f"..//data//{activity_name}.csv", index = False)

Lagunilla Hiking Bajada
Hike


KeyError: 0